In [ ]:
## set covid19_datasciente as path for python find bulletin package
import sys
from pathlib import Path
from os import getcwd
sys.path.append(str(Path(getcwd()).parent.parent)) ## ../../covid19datascience* <- set parent level here

In [ ]:
from os.path import join
import pandas as pd
pd.set_option('display.max_columns', None)
from sys import exit
from datetime import date, timedelta

In [ ]:
from bulletin.systems.casos_confirmados import CasosConfirmados
from bulletin.systems.tb_pacientes import TbPacientes
from bulletin.utils.normalize import normalize_hash, normalize_labels, normalize_text, date_hash
from bulletin.utils.static import Municipios
from bulletin.utils.xls_ import fit_cols
from bulletin import default_input

In [ ]:
municipios = Municipios()
municipios['mun_resid'] = municipios['municipio'].apply(normalize_text)

In [ ]:
tb_pacientes = TbPacientes()
tb_pacientes.update()
casost = tb_pacientes.get()
# casost = pd.read_pickle('tb_pacientes.pkl')#tb_pacientes.get()
casost['nome'] = casost['nome'].apply(lambda nome: ''.join(filter(lambda x: ('A' <= x <= 'Z') or ( x == ' '), nome)))
casost = casost.sort_values(['dt_notificacao','nome','idade'])
print(f"casost len: {len(casost)}\n")
casost.loc[casost['obito']=='Sim','obito'] = 'SIM'
casost.loc[casost['obito']!='SIM','obito'] = 'NAO'
print(f"casost len: {len(casost)}\n")
casost.groupby('obito')[['sexo']].count()

In [ ]:
casost

In [ ]:
casost.groupby('status')[['sexo']].count()

In [ ]:
casosct = casost.copy()

casosct['evolucao'] = 3
casosct.loc[casosct['obito']=='SIM','evolucao'] = 2
casosct.loc[(casosct['obito']!='SIM')&(casosct['status'].notna()),'evolucao'] = 1

In [ ]:
casosct.loc[casosct['obito']=='SIM','data_evolucao'] = casosct.loc[casosct['obito']=='SIM','dt_obito'] 
casosct.loc[(casosct['obito']!='SIM')&(casosct['status'].notna()),'data_evolucao'] = casosct.loc[(casosct['obito']!='SIM')&(casosct['status'].notna()),'dt_atualizacao']

In [ ]:
casosct.loc[casosct['obito']=='SIM','data_com_evolucao'] = casosct.loc[casosct['obito']=='SIM','dt_com_obito'] 
casosct.loc[(casosct['obito']=='SIM')&(casosct['data_com_evolucao'].isna()),'data_com_evolucao'] = casosct.loc[(casosct['obito']=='SIM')&(casosct['data_com_evolucao'].isna()),'dt_obito']
casosct.loc[(casosct['obito']!='SIM')&(casosct['status'].notna()),'data_com_evolucao'] = casosct.loc[(casosct['obito']!='SIM')&(casosct['status'].notna()),'dt_com_recuperado'] 

In [ ]:
casosct.groupby('evolucao')[['sexo']].count()

In [ ]:
casosct['uf_resid'] = 'PR'
casosct['ibge_res_pr'] = casosct['ibge_res_pr'].apply(int)

casosct.loc[casosct['ibge_res_pr']==999999,'uf_resid'] = casosct.loc[casosct['ibge_res_pr']==999999,'mun_resid'].apply(lambda x: x.split('/')[-1])
casosct.loc[casosct['ibge_res_pr']==999999,'mun_resid'] = casosct.loc[casosct['ibge_res_pr']==999999,'mun_resid'].apply(lambda x: x.split('/')[0])

casos_fora = pd.merge(casosct.loc[casosct['ibge_res_pr']==999999].drop(columns=['ibge_res_pr']),municipios[['ibge','mun_resid','uf']].rename(columns={'uf':'uf_resid','ibge':'ibge_res_pr'}), on=['mun_resid','uf_resid'], how='left')
casos_fora.loc[casos_fora['ibge_res_pr'].isna(),'ibge_res_pr'] = 999999
casos_fora['ibge_res_pr'] = casos_fora['ibge_res_pr'].astype(int)
casos_fora = casos_fora.set_index('identificacao')
casosct = casosct.set_index('identificacao')
casosct.update(casos_fora)

casosct = casosct.reset_index()

casosct['ibge_res_pr'] = casosct['ibge_res_pr'].astype(int)
casosct['idade'] = casosct['idade'].astype(int)

casosct.loc[casosct['uf_resid']!='PR']

In [ ]:
casosct.groupby(['evolucao'])[['sexo']].count()

In [ ]:
casosct['id_notifica'] = -1
casosct['ibge_resid'] = casosct['ibge_res_pr']
casosct['ibge_atend'] = casosct['ibge_atend_pr'].apply(int)
casosct['comunicacao'] = casosct['dt_notificacao']
casosct['is'] = casosct['dt_inicio_sintomas']

casosct = casosct[['identificacao', 'id_notifica', 'uf_resid', 'ibge_resid', 'ibge_atend', 'nome', 'sexo', 'idade', 'laboratorio', 'dt_diag', 'comunicacao', 'is', 'evolucao', 'data_evolucao', 'data_com_evolucao']]
casosct

In [ ]:
casos_confirmados = casosct.copy()
casos_confirmados

In [ ]:
casos_confirmados.dtypes

In [ ]:
casos_confirmados.loc[casos_confirmados['is'].notna(),'is'] = casos_confirmados.loc[casos_confirmados['is'].notna(),'is'].apply(str).str[:10]
casos_confirmados['is'] = pd.to_datetime(casos_confirmados['is'],format='%d/%m/%Y',errors='ignore')
casos_confirmados['is'] = pd.to_datetime(casos_confirmados['is'],format='%Y-%m-%d',errors='coerce')
casos_confirmados['is']

In [ ]:
casos_confirmados.loc[casos_confirmados['data_evolucao'].notna(),'data_evolucao'] = casos_confirmados.loc[casos_confirmados['data_evolucao'].notna(),'data_evolucao'].apply(str).str[:10]
casos_confirmados['data_evolucao'] = pd.to_datetime(casos_confirmados['data_evolucao'],format='%d/%m/%Y',errors='ignore')
casos_confirmados['data_evolucao'] = pd.to_datetime(casos_confirmados['data_evolucao'],format='%Y-%m-%d',errors='coerce')
casos_confirmados['data_evolucao']

In [ ]:
casos_confirmados['evolucao'] = casos_confirmados['evolucao'].astype(int)
casos_confirmados.dtypes

In [ ]:
casos_confirmados['hash'] = ( casos_confirmados['nome'].apply(normalize_hash) +
               casos_confirmados['idade'].astype(int).astype(str) +
               casos_confirmados['ibge_resid'].astype(int).astype(str) )

casos_confirmados['hash_less'] = ( casos_confirmados['nome'].apply(normalize_hash) +
                    casos_confirmados['idade'].apply(lambda x: str(int(x-1))) +
                    casos_confirmados['ibge_resid'].astype(int).astype(str) )

casos_confirmados['hash_more'] = ( casos_confirmados['nome'].apply(normalize_hash) +
                    casos_confirmados['idade'].apply(lambda x: str(int(x+1))) +
                    casos_confirmados['ibge_resid'].astype(int).astype(str) )

casos_confirmados['hash_atend'] = (casos_confirmados['nome'].apply(normalize_hash) +
                    casos_confirmados['idade'].astype(int).astype(str) +
                    casos_confirmados['ibge_atend'].astype(int).astype(str) )

casos_confirmados['hash_less_atend'] = (casos_confirmados['nome'].apply(normalize_hash) +
                    casos_confirmados['idade'].apply(lambda x: str(int(x-1))) +
                    casos_confirmados['ibge_atend'].astype(int).astype(str) )

casos_confirmados['hash_more_atend'] = (casos_confirmados['nome'].apply(normalize_hash) +
                    casos_confirmados['idade'].apply(lambda x: str(int(x+1))) +
                    casos_confirmados['ibge_atend'].astype(int).astype(str) )

casos_confirmados['hash_diag'] = ( casos_confirmados['nome'].apply(normalize_hash) +
                    casos_confirmados['dt_diag'].apply(date_hash) )

In [ ]:
casos_confirmados = casos_confirmados[['identificacao','id_notifica','uf_resid','ibge_resid','ibge_atend','nome','sexo','idade','laboratorio','dt_diag','comunicacao','is','evolucao','data_evolucao','data_com_evolucao','hash','hash_less','hash_more','hash_atend','hash_less_atend','hash_more_atend','hash_diag']]

In [ ]:
casos_confirmados.columns = ['identificacao','id_notifica','uf_residencia','ibge_residencia','ibge_unidade_notifica','paciente','sexo','idade','exame','data_diagnostico','data_comunicacao','data_1o_sintomas','evolucao','data_evolucao','data_com_evolucao','hash_resid','hash_resid_less','hash_resid_more','hash_atend','hash_atend_less','hash_atend_more','hash_diag']

In [ ]:
casos_confirmados.loc[casos_confirmados['uf_residencia']!='PR']

In [ ]:
casos_confirmados

In [ ]:
cca = CasosConfirmados()
cca.df = casos_confirmados
cca.save('cca_05_08_2021',replace=True,compress=False)

In [ ]:
cca.database